In [ ]:
import pandas as pd
from deltalake import write_deltalake, DeltaTable
import pyarrow.dataset as ds
import os
from boto3 import Session
import boto3

In [ ]:
os.environ["AWS_PROFILE"] = "defiland"
os.environ["AWS_DEFAULT_REGION"] = "eu-central-1"

In [ ]:
import numpy as np

# Set the start and end dates for the data
start_date = "2024-01-01"
end_date = "2024-01-02"

# Generate a date range with 30-minute intervals
date_range = pd.date_range(start=start_date, end=end_date, freq='30T')

# Generate random OHLCV data
np.random.seed(42) 
data = {
    "timestamp": date_range,
    "date": date_range.date,
    "open": np.random.uniform(low=30000, high=40000, size=len(date_range)),
    "high": np.random.uniform(low=30000, high=40000, size=len(date_range)),
    "low": np.random.uniform(low=30000, high=40000, size=len(date_range)),
    "close": np.random.uniform(low=30000, high=40000, size=len(date_range)),
    "volume": np.random.uniform(low=0, high=10, size=len(date_range))
}

# Create a DataFrame
df = pd.DataFrame(data)

# Ensure that high is always greater than or equal to low
df["high"] = df[["open", "high", "low", "close"]].max(axis=1)
df["low"] = df[["open", "high", "low", "close"]].min(axis=1)

In [ ]:
session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()
storage_options = {}
storage_options["AWS_ACCESS_KEY_ID"] = current_credentials.access_key
storage_options["AWS_SECRET_ACCESS_KEY"] = current_credentials.secret_key
storage_options["AWS_S3_ALLOW_UNSAFE_RENAME"] = "true"
storage_options["AWS_REGION"] = "eu-central-1"

In [ ]:
storage_options

In [ ]:
write_deltalake("s3://fridon-ai-coin-data/coin-data-table-2/", df, partition_by=["date"],  mode="append", storage_options=storage_options)

In [ ]:
s3 = boto3.client('s3')
response = s3.list_objects_v2(Bucket='fridon-ai-coin-data', Prefix='coin-data-table-2/')
for obj in response.get('Contents', []):
    print(obj['Key'])

In [ ]:
s3fs = S3FileSystem(region="eu-central-1", access_key=current_credentials.access_key, secret_key=current_credentials.secret_key)

In [ ]:
table = DeltaTable("s3://fridon-ai-coin-data/coin-data-table-1/", storage_options=storage_options)


In [ ]:
table.optimize.compact()

In [ ]:
table.files()

In [ ]:
df = table.to_pyarrow_dataset()

In [ ]:
df

In [ ]:
table.vacuum(retention_hours=0, enforce_retention_duration=False, dry_run=False)


In [ ]:
table.optimize.z_order(["a", "created_at", "b"])

In [ ]:
table.vacuum(retention_hours=0, enforce_retention_duration=False, dry_run=False)

In [ ]:
table.create_checkpoint()

In [ ]:
table.to_pandas(columns=["a", "b", "created_at"], filters=(ds.field("a") > 1) & (ds.field("b") > 1) & (ds.field("created_at") < pd.to_datetime("2022-01-03")))

In [ ]:
table.schema()

In [ ]:
from libs.repositories import coins_delta_lake_repository

In [ ]:
table = coins_delta_lake_repository.read_table("ohlcv_raw")
table 


In [ ]:
from apps.fridon_crones.main import generate_dummy_data

In [ ]:
data = generate_dummy_data()

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

In [ ]:
coins_delta_lake_repository.write_to_table("ohlcv_raw", df, mode="append")

In [ ]:
table_raw = coins_delta_lake_repository.read_table("ohlcv_raw")
table_1h = coins_delta_lake_repository.read_table("ohlcv_1h")
table_4h = coins_delta_lake_repository.read_table("ohlcv_4h")
# table_1d = coins_delta_lake_repository.read_table("ohlcv_1d")
# table_1w = coins_delta_lake_repository.read_table("ohlcv_1w")



In [ ]:
table_1h

In [ ]:
table_4h

In [ ]:
table_raw

In [ ]:
import polars as pl

In [ ]:
table_raw.filter(pl.col("coin") == "BTC").get_column("volume").sum()

In [3]:
import requests

url = "https://private-api.birdeye.so/defi/ohlcv?address=So11111111111111111111111111111111111111112&type=15m"

headers = {
    "accept": "application/json",
    "x-chain": "solana",
    "X-API-KEY": "c130534e155a4732ad86dee0a338555f"
}

response = requests.get(url, headers=headers)

print(response.text)

ConnectionError: HTTPSConnectionPool(host='private-api.birdeye.so', port=443): Max retries exceeded with url: /defi/ohlcv?address=So11111111111111111111111111111111111111112&type=15m (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x107d38810>: Failed to resolve 'private-api.birdeye.so' ([Errno 8] nodename nor servname provided, or not known)"))